# Assignment 1
## RSA signatures task (2)
### Malthe Faurschou Tøttrup au644177

Problem context:

In this task we want to show the grade 12 to a server. The professor has made a server that can give grades automatically. Once a grade is given it can be sendt back to the server in form of a cookie to be verified. The issue is that the server will never give a grade that is higher than 10. Therefore, we need to construct this grade message ourselves.

Since the source code for the server is given, we can see what the message the server expects for the grade 12 looks like. However, the server uses RSA to verify grade messages, and therefore we cannot simply send this message to the server without a valid signature to go with it.

The server provides a service for signing messages. However, this service is smart enough to detect messages containing grade information such as '12', and will reject the request. Therefore it is neccessary to bypass this message filter.

The attack:

We want to send message $m$ with valid signature $s$ to the server in form of a cookie. However, $s$ cannot be obtained directly and will need to be forged from two different signatures $s_1$ and $s_2$. These will need to be obtained through the signing of two randomly looking messsages $m_1$ and $m_2$. However, the following relationship must be true: 

$m = m_{1}m_{2} \space \texttt{mod} \space N$

We can obtain signatures $s_1$ and $s_2$ by sending $m_1$ and $m_2$ to the signature generating service. Once we have $s_1$ and $s_2$ we can calculate $s$ as follows:

$s = s^e \space \texttt{mod} \space N = s_{1}s_{2} \space \texttt{mod} \space N$

Once we have $s$ we can convince the server that our message $m$ is valid and we did indeed get a 12. 


In [53]:
# import libraries

import requests
import json
import random
import base64

In [54]:
srv_url = 'http://localhost:5000' # server url
# netsec_ip = '192.168.2.0/24'
netsec_url = 'https://cbc-rsa.syssec.dk:8001/'

# get public key and N from server

res = requests.get(netsec_url + '/pk', verify=False)
j = json.loads(res.text)
N = j['N']
e = j['e']

# compute private key
# d = pow(e, -1, phi(N))

/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [55]:
# figureing out string to hex to int to hex to string gymnstics 

m = 'You got a 12 because you are an excellent student! :)'

m_hex = m.encode('utf-8').hex() # convert to hex

print(m_hex)

m_string = bytes.fromhex(m_hex) # convert back to string

print(m_string) 

m_int = int.from_bytes(m_string, 'big') # convert to int

print(m_int)

m_int_hex = m_int.to_bytes((m_int.bit_length() + 7) // 8, 'big').hex() # convert back to hex

print(m_int_hex)

m_int_string = bytes.fromhex(m_int_hex) # convert back to string

print(m_int_string)


596f7520676f742061203132206265636175736520796f752061726520616e20657863656c6c656e742073747564656e7421203a29
b'You got a 12 because you are an excellent student! :)'
15135178855545273522390445503983425087237773086288803041518748436246698994070655615214064723425547390585341561560870852814191145
596f7520676f742061203132206265636175736520796f752061726520616e20657863656c6c656e742073747564656e7421203a29
b'You got a 12 because you are an excellent student! :)'


In [56]:
# generate random looking messages m1 and m2

m1 = random.randint(1, 100) # pick random starting number between 1 and 100

# loop until m1 is a factor of m_int
while(m_int % m1 != 0):
    m1 += 1

# calculate m2
m2 = m_int // m1 % N

# check if m1 * m2 is equal to m_int % N 
if (m1 * m2) == m_int % N:
    print('works')
else:
    print('doesnt work')

works


In [57]:
# sign m1 and m2

m1_hex = m1.to_bytes((m1.bit_length() + 7) // 8, 'big').hex() # convert to hex

m1_res = requests.get(f'{netsec_url}/sign_random_document_for_students/{m1_hex}',  verify=False)

m1_j = json.loads(m1_res.text)
s1 = m1_j['signature']


m2_hex = m2.to_bytes((m2.bit_length() + 7) // 8, 'big').hex() # convert to hex

m2_res = requests.get(f'{netsec_url}/sign_random_document_for_students/{m2_hex}',  verify=False)

m2_j = json.loads(m2_res.text)
s2 = m2_j['signature']

# check signatures
print(str(m1_j['signature']))
print(str(m2_j['signature']))


/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  w

77a47396fdbe026fef2d8a5673849d37e6531a2e2965f5ac42e6de2140718ea9338a04785f9967d38f780ee37eb8e9dde678197e9a6a4701b4e4af8087188adcbfd0d230ecc4ad69f7a7d7222ce4aa663fdee227c29c3c3f9355fd66a83095da22db8b4b80d05befb1fbafb1cf6c6ea7d871e8aa8e98f0b3fce80d3bf17b213e33709a1e48c1d49fad5972835bd03003e0058176892c81d090fe1c72b34566700a01cbb88f4606f4e7bf060b699c18c1694fd8813e153870da0a86d815bd642d7230843d0d3146c8a9029b078710858f8df0c41d6caf5c849ae813b00bb4b2ec5221fc9768a77db58e4dd12af126193e600d97db27477e6ff215c2e51a55b3acb54b5ec930caf1e38703ce570297ae8daee0d19f8e37e1b63c333a58beb69b91e55a2589dcc891269bce3a74445c23cee4d81eba7741b78a6c2622c039d5044107f6b17b6b1be19b20985f9641cbbfa3592f9ff6232d1a548fa3283e231ea8fb40eb2b11cfbcf2e4792fe4e7a9a2c72efbaa041b59439bff6c732bb941aec3e0
53a86ee4cd2a9da22e5c3e7bcd2ad24139976be5f9d5edac39b3297c2d2d808e45ad804f93f898813ab64503815bd88bf70dff22f53d69e17504d3b7f0164776525b9081c5b9161201a6a1c59963e905684d9609d2b70bc8db57b9d0a37310b4cffc8e423be1dd766edccc36a0fc9ba69d2e4a6

In [58]:
# compute signature for m
s1_int = int.from_bytes(bytes.fromhex(s1), 'big')
s2_int = int.from_bytes(bytes.fromhex(s2), 'big')

s = (s1_int * s2_int) % N

s_hex = s.to_bytes((s.bit_length() + 7) // 8, 'big').hex() # convert to hex

print(s_hex)

7e29757d786ab749003816cd81c505f451019ee13cf13c9ebbffa329e3f71a079f906c3766a850cd6567f835f304b6ac1f9cbc319ff71904ca21d2c927b15be6b5a3d5643f158b5c815eb2f05e18683eee57942d5c34cffa35a327eb79144ed0b0b450a5a4b146084e5a6a2657cba4606752061a4f731e49e833c96348df99e4ef29262a8621ac7bcf56136fc39efc7ed8096fbc0d4c94e4fecf975c98174acd06b7f82ac201b73c189f255d5bc697073e058cd960ccc0aefd0a791bbd884df3eb568b4dd4bdc0a5c1e22cdde6eab7350aa042e9deb2eed0a2af0c99f06ecc9516098381bcd77b3c2d27cf95823268b6a81b6de0af0bcae2706f1aa66002b0dd8c93138b6ecc666cc0cffce19b4425d0ea52b95391c735a5009dc5657f8d4b8d5d9127f915e1dc3439d6747cb4d53e69b61be4d3324ab8876ddcc2bc9d45b34b80ba6b4ceb886d4b8d33e18f251669822bc6312d2b8094482d96233a4a4fb8e4de1ccb423e1f95e107b241198f864e3bf99879ef9c60a78938dc416bc92d1d23


In [59]:
# send the grade message and signature

grade = base64.b64encode(json.dumps({'msg': m_hex, 'signature': s_hex}).encode()).decode() # thanks CoPilot

res = requests.get(f'{netsec_url}/grade', cookies={'grade': grade},  verify=False)

print(res.text) # celebration!

<p>You got a 12 because you are an excellent student! :)</p>


/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [60]:
# recieve a quote

res = requests.get(f'{netsec_url}/quote', cookies={'grade': grade},  verify=False)

res.text # hmm interesting quote 

/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/malthet/anaconda3/envs/netsec/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbc-rsa.syssec.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'<quote>\nWho needs a wrench if there are so many ways to break RSA? (cf. https://xkcd.com/538/)\n</quote>'